In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-08-18


Found rundate 2024-08-17 10:00:00 < 2024-08-18 at commit 2c39370b; returning commit 0e1beb2


'2024-08-18 10:00:04'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-08-18 10:00:04, NJSP was reporting 392 YTD deaths
Current YTD Deaths (2025-08-18 10:00:01-04:00): 350
Previous year YTD Deaths (adjusted; 2024-08-18 10:00:04): 391.36767833465746
Projected 2025 total: 624.1278904568553
-10.6% change


00edd539b92d4ed0010961d0cc8424dec369b9d4: FAUQStats cache miss: 2024, Sun Aug 18 10:00:04 EDT 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,STREET,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,HIGHWAY,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,NaN,State Highway 1,1.0,0.0,0.0,1.0,0.0,1,NaN,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,NaN,State Highway 129,1.0,1.0,0.0,0.0,0.0,129,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,NaN,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,446,0.0,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,East Oak St,East Oak St,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-05 14:12:00-05:00
13362,07,Essex,0701,Belleville Town,NaN,County 647 MP .76,1.0,0.0,0.0,1.0,0.0,647,NaN,2024-01-05 18:08:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13359,12,Middlesex,1214,New Brunswick City,Townsend St,Townsend St,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-08-15 00:48:00-04:00
13360,18,Somerset,1808,Franklin Twsp,Manville Cswy,Manville Cswy,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-08-15 19:36:00-04:00
13364,12,Middlesex,1217,Piscataway Twsp,NaN,Interstate 287 MP 8.8,1.0,1.0,0.0,0.0,0.0,287,NaN,2024-08-15 20:24:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,14,2,6,0,21
Bergen,12,1,4,0,17
Burlington,15,4,4,0,22
Camden,13,2,8,1,23
Cape May,7,3,0,0,9
Cumberland,6,2,3,0,10
Essex,8,3,17,1,29
Gloucester,14,4,2,0,17
Hudson,4,1,3,1,8


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

62.8% through the year, 37.2% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,18,31,14
Bergen,driver,12,14,12
Burlington,driver,20,31,15
Camden,driver,12,19,13
Cape May,driver,3,8,7
...,...,...,...,...
Salem,crashes,9,14,10
Somerset,crashes,10,18,7
Sussex,crashes,8,12,3


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,40,0,25,4,12
Bergen,28,0,14,2,11
Burlington,36,0,24,4,9
Camden,33,1,20,3,10
Cape May,23,0,16,4,0
Cumberland,22,0,13,6,8
Essex,45,1,15,10,25
Gloucester,44,0,32,14,10
Hudson,23,1,9,1,19


In [9]:
projected.to_csv(PROJECTED_CSV)